In [ ]:
import header
from header import __root__
from src import gs

In [ ]:
import pandas as pd
import zipfile
from pathlib import Path
from typing import Optional, List, Dict, Any
from types import SimpleNamespace
from src.utils.jjson import j_loads, j_loads_ns
from src.utils.file import read_text_file, recursively_yield_file_path

In [ ]:
class Config:
    """Класс конфигурации скрипта."""
    ENDPOINT: Path = __root__ / 'SANDBOX' / 'davidka'
    config: SimpleNamespace = j_loads_ns(ENDPOINT / 'davidka.json')
    STORAGE: Path
    try:
        STORAGE = Path(config.storage)
    except Exception as ex:
        raise ValueError(f"Ошибка загрузки пути к хранилищу: ", ex)
        
    TRAIN_STORAGE:Path = Path(config.train_storage)
    UZZIPPED_STORAGE:Path = Path(config.unzipeped_storage)
    GEMINI_API_KEY: Optional[str] = gs.credentials.gemini.onela.api_key # Установить имя владельца ключа (onela, kazazrinov, ...)
    GEMINI_MODEL_NAME: str = 'gemini-2.0-flash-exp' # Актуальное имя модели может отличаться, например, 'gemini-1.5-flash-latest'
    system_instructuction: str | None = read_text_file(ENDPOINT / 'instructions/analize_html.md')
    updated_links_file_name: str = 'updated_links.json'
    DELAY_AFTER_LINK_PROCESSING: int = 15
    WINDOW_MODE: str = 'headless'
    
    

In [ ]:
"""
with zipfile.ZipFile("your_file.zip", 'r') as zip_ref:
    zip_ref.extractall(ENDPOINT / 'extracted_data')  # например, 'data'
    """

In [ ]:
#print(train_files)
products_dict:dict = {}
for file in recursively_yield_file_path(Config.TRAIN_STORAGE,'*.json'):
    file_dict = j_loads(file)
    products_dict.update(file_dict)


In [ ]:
print(len(products_dict))


In [ ]:
df = pd.DataFrame(products_dict)

In [ ]:
print(df.head(2))

In [ ]:
print(df.columns[:5])

In [ ]:
df_transposed = df.T

In [ ]:
print(df_transposed.columns)

In [ ]:
df_cleaned = df_transposed.dropna(subset=['supplier'])


In [ ]:
supplier_list = df_cleaned['supplier'].dropna().unique()
print(supplier_list)

In [ ]:
supplier_counts = df_clean.groupby('supplier').count().reset_index()

In [ ]:
print(supplier_counts)

In [ ]:
df_transposed.columns

## В следующей функции возникает ошибка скорее всего из-за неправильного символа UNICODE в названии столбца

In [ ]:
columns_to_check = ['category']
actual_category_column_name = None
for c_name in df_transposed.columns:
    if 'category' in c_name: # или c_name == 'category'
        actual_category_column_name = c_name
        break

if actual_category_column_name:
    columns_to_check = [actual_category_column_name]
else:
    columns_to_check = [] # 'category' не найдена

existing_columns = [col for col in columns_to_check if col in df_transposed.columns]


# Если колонки существуют, считаем товары
if existing_columns:
    # Объединяем значения из существующих колонок
    category_counts = pd.concat([df[col] for col in existing_columns])

    # Считаем количество товаров в каждой категории
    category_counts = category_counts.dropna().value_counts().reset_index()
    category_counts.columns = ['category', 'product_count']
else:
    category_counts = pd.DataFrame(columns=['category', 'product_count'])

print(category_counts)

In [41]:
df_transposed.to_csv(Config.TRAIN_STORAGE / 'tarin.csv', index=False, encoding='utf-8')